# Practical 1

In [ ]:
# Practical 1
''' 
Aim:
    Apply data cleaning techniques on any dataset (e.g., Paper Reviews dataset in UCI repository).
    Techniques may include:
    1. handling missing values, 
    2. outliers and inconsistent values. 
    3. A set of validation rules can be prepared based on the dataset and validations can be performed.
'''

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student-mat.csv"
df = pd.read_csv(url, sep=';')
print("Dataset loaded successfully!")
print(df.head())

                                               paper
0  {'id': 1, 'preliminary_decision': 'accept', 'r...
1  {'id': 2, 'preliminary_decision': 'accept', 'r...
2  {'id': 3, 'preliminary_decision': 'accept', 'r...
3  {'id': 4, 'preliminary_decision': 'accept', 'r...
4  {'id': 5, 'preliminary_decision': 'accept', 'r...


In [2]:
# 1. Basic overview
print(df.shape)
print(df.dtypes)
print(df.isna().sum())

(172, 1)
paper    object
dtype: object
paper    0
dtype: int64


In [3]:
# 2. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [4]:
# 3. Drop exact duplicate rows
df = df.drop_duplicates()

In [5]:
# 4. Validate and fix paper_id
# If paper_id should be int, attempt conversion; otherwise keep string
df['paper_id'] = df['paper_id'].astype(str).str.strip()
# flag suspicious IDs
bad_ids = df[df['paper_id'].str.len() == 0]
print("Bad paper ids:", len(bad_ids))

KeyError: 'paper_id'

In [ ]:
# 5. Parse dates
def safe_parse_date(x):
    try:
        return pd.to_datetime(x, errors='coerce')
    except:
        return pd.NaT

df['submission_date'] = df['submission_date'].apply(safe_parse_date)
# Rule: submission_date between 1990-01-01 and today
mask = (df['submission_date'] < pd.Timestamp('1990-01-01')) | (df['submission_date'] > pd.Timestamp.now())
print("Out-of-range dates:", mask.sum())

In [ ]:
# 6. Score normalization and missing handling
# If score is string like "8/10" or "8.0", normalize
def parse_score(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float)): return float(x)
    s = str(x).strip()
    if '/' in s:
        a,b = s.split('/')
        try:
            return float(a)/float(b) * 10  # normalize to scale-10
        except:
            return np.nan
    try:
        return float(s)
    except:
        return np.nan

df['score'] = df['score'].apply(parse_score)
# Rule: acceptable scores 0..10 (or your dataset's range)
df.loc[(df['score'] < 0) | (df['score'] > 10), 'score'] = np.nan


In [ ]:
# 7. Recommendation cleanup
allowed = {'accept','reject','weak-accept','weak-reject','borderline','revise'}
df['recommendation'] = df['recommendation'].astype(str).str.lower().str.strip()
df.loc[~df['recommendation'].isin(allowed), 'recommendation'] = np.nan

In [ ]:
# 8. Text cleaning: trim whitespace and normalize encoding
for col in ['title','abstract','review_text','authors']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace({'nan': pd.NA})



In [ ]:
# 9. Missing value strategy (choices)
# - If score missing but recommendation present, you might impute score based on recommendation median
# - If essential fields (paper_id, reviewer_id) missing -> drop
df = df.dropna(subset=['paper_id','reviewer_id'])

# Example: impute scores by recommendation median
if 'recommendation' in df.columns:
    medians = df.groupby('recommendation')['score'].median()
    def impute(row):
        if pd.isna(row['score']) and pd.notna(row['recommendation']):
            return medians.get(row['recommendation'], np.nan)
        return row['score']
    df['score'] = df.apply(impute, axis=1)



In [ ]:
# 10. Outlier detection for numeric fields (score)
# Use IQR method
Q1 = df['score'].quantile(0.25)
Q3 = df['score'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
outliers = df[(df['score'] < lower) | (df['score'] > upper)]
print("Score outliers:", len(outliers))

# Option: keep outliers but flag them
df['score_outlier'] = df['score'].apply(lambda x: x < lower or x > upper)



In [ ]:
# 11. Save cleaned dataset for reproducibility
df.to_csv('paper_reviews_cleaned.csv', index=False)
print("Cleaned saved:", df.shape)
